<a href="https://colab.research.google.com/github/ftmthb/NER/blob/main/gradio_ner_pfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### installations

In [ ]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 28.7 MB/s eta 0:00:00


In [ ]:
!virtualenv /content/drive/MyDrive/virtual_env


created virtual environment CPython3.10.12.final.0-64 in 2312ms
  creator CPython3Posix(dest=/content/drive/MyDrive/virtual_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.1.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.6 MB/s eta 0:00:00


In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install ipymarkup

  Preparing metadata (setup.py) ... done
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=f17abf528f1a0c5f8b71ef57623ee800c8849496a2e8f3845993a56b7167e059
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


##### env activate

In [1]:
import sys
sys.path.append("/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages")

#### import model

In [2]:
from transformers import pipeline

classifierx = pipeline('ner',r"Fatimetou/xlm-roberta-large",  device_map = 'auto')

#### Functions

##### id_2_label

In [3]:
label_2_id = {'O': 0,
 'B-FULL_NAME': 1,
 'I-FULL_NAME': 2,
 'B-CIN': 3,
 'I-CIN': 4,
 'B-PROPERTY_TITLE': 5,
 'I-PROPERTY_TITLE': 6,
 'B-PROPERTY_PRICE': 7,
 'I-PROPERTY_PRICE': 8,
 'B-SAVE_DATE': 9,
 'I-SAVE_DATE': 10}

label_2_id = {v: k for k, v in label_2_id.items()}

#### clean text
from redundant characters, whitespaces

In [4]:
import re
"""
  bert handles whitespaes while roberta treats them as separate tokens, while giving them the G tokewhich also comes at the beggining of other splitted words

  this function replaces all the unwanted or redundant characters with a space

  it also deals with short lines and therefore

"""

def clean_text_data(input_text):

  replacements = [
      (r'٨{4,}', ' ',),
      (r'\s{2,}', ' ',),
      (r'\n{2,}', r'\n',),
      (r'\.{2,}', ' '),
      (r'\={2,}', ' '),
      (r'\-{2,}', ' '),
      ('x{2,}', ' '),
      (r'\*{2,}', ' '),
      (r'\xad', ' '),
      (r'\xa0', ' '),
      (r'\x80', ''),
      (r'\x9d', ''),
      (r'\x13', ' '),
      ]

  for old, new in replacements:
      input_text = re.sub(old, new, input_text)

  lines = input_text.splitlines()

  lines = [re.sub(r'\s+', ' ', line.strip()) for line in lines if line.strip()]

  clean_lines = []
  current_line = ""

  for line in lines:

      if current_line  and not current_line.endswith(('.', ':')):   #and len(current_line) + len(line) < 700
          current_line += " " + line
      else:
          if current_line:
              clean_lines.append(current_line)
          current_line = line

  if current_line:
      clean_lines.append(current_line)

  return "\n".join(clean_lines)

### Split text

In some cases where the given text is tokenized into over 512 tokens, it needs to be splitted into chunks  

In [5]:
def split_text(text, max_length=512):
    if len(text) <= max_length:
        return [(0, text)]

    chunks = []
    start = 0
    while start < len(text):
        end = start + max_length
        if end >= len(text):
            chunks.append((start, text[start:]))
            break
        chunks.append((start, text[start:end]))
        start = end
    return chunks       #where chunk is a list of tuples with the index of the first 'string' of the chunk and the text composing it in the second position

#### process chunk

as in classify the NE

In [6]:
def process_chunk(chunk_start, chunk_text):
    res = classifierx(chunk_text)

    spans = []  #for the ipymarkup functions
    for elt in res:
        label = elt['entity']
        id = int(label.split('_')[1])
        if id != 0:
            spans.append((
                chunk_start + elt['start'],
                chunk_start + elt['end'],
                label_2_id[id]
            ))
    return spans    #classifies ie extracts named entities by chunk and return spans like [(pos_start, pos_end, label), ...]

#### custom palette

adjust the box_markup function to give different colors for each label

In [7]:
from ipymarkup import show_span_box_markup, format_span_box_markup

from ipymarkup.palette import palette, Color, BLUE, RED, GREEN, ORANGE, PURPLE, BROWN
from ipymarkup.palette import material


TEAL = Color(
    'teal',
    background=material('Teal', '50'),
    border=material('Teal', '100'),
    text=material('Teal', '300'),
    line=material('Teal', '200')
)

AMBER = Color(
    'amber',
    background=material('Amber', '50'),
    border=material('Amber', '100'),
    text=material('Amber', '300'),
    line=material('Amber', '200')
)

PINK = Color(
    'pink',
    background=material('Pink', '50'),
    border=material('Pink', '100'),
    text=material('Pink', '300'),
    line=material('Pink', '200')
)

CYAN = Color(
    'cyan',
    background=material('Cyan', '50'),
    border=material('Cyan', '100'),
    text=material('Cyan', '300'),
    line=material('Cyan', '200')
)

YELLOW = Color(
    'yellow',
    background=material('Yellow', '50'),
    border=material('Yellow', '100'),
    text=material('Yellow', '300'),
    line=material('Yellow', '200')
)

def get_span_box_markup(text, spans):
    custom_palette = palette(
        **{
            "B-FULL_NAME": BLUE, "I-FULL_NAME": CYAN,
            "B-CIN": RED, "I-CIN": PINK,
            "B-PROPERTY_TITLE": GREEN, "I-PROPERTY_TITLE": TEAL,
            "B-PROPERTY_PRICE": ORANGE, "I-PROPERTY_PRICE": YELLOW,
            "B-SAVE_DATE": PURPLE, "I-SAVE_DATE": AMBER,
            "B-OWNER_NAME": BROWN, "I-OWNER_NAME": BLUE,
            "B-PROPERTY_AREA": CYAN, "I-PROPERTY_AREA": RED,
            "B-TRANSACTION_DATE": TEAL, "I-TRANSACTION_DATE": PINK
        }
    )
    return ''.join(format_span_box_markup(text, spans, palette=custom_palette))


####  predict function

In [8]:
import concurrent.futures

def predict(text_to_classify):
    text_to_classify = clean_text_data(text_to_classify)
    chunks = split_text(text_to_classify)

    with concurrent.futures.ThreadPoolExecutor() as executor:   #chunk parallel processing (time)
        future_to_chunk = {executor.submit(process_chunk, start, text): (start, text) for start, text in chunks}
        all_spans = []
        for future in concurrent.futures.as_completed(future_to_chunk):
            all_spans.extend(future.result())

    # return show_span_box_markup(text_to_classify, all_spans)    #for in notebook

    return get_span_box_markup(text_to_classify, all_spans)         #for interface


#### interface configuration

In [10]:
import gradio as gr
iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=100, label="Texte à classifier"),
    outputs=gr.HTML(label="Résultat"),
    title="Système de Reconnaissance des entités nommées (pour longs textes)",
    description="Entrez le texte:"
)

iface.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e1ae8572b684a2eb50.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e1ae8572b684a2eb50.gradio.live


In [12]:
import gradio as gr
iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=100, label="Texte à classifier"),
    outputs=gr.HTML(label="Résultat"),
    title="Système de Reconnaissance des entités nommées (pour longs textes)",
    description="Entrez le texte:"
)

In [13]:
iface.launch(share=True, debug = True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9708544f9d15e994ed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9708544f9d15e994ed.gradio.live
